In [1]:
import subprocess
import re
from multiprocessing import Pool
from time import sleep

In [2]:
poll_command = ['/usr/local/bin/ibmcloud', 'fn', 'activation', 'poll']

In [3]:
activation_list = ['/usr/local/bin/ibmcloud', 'fn', 'activation', 'list']
invoke_activation = ['/usr/local/bin/ibmcloud', 'fn', 'action', 'invoke', 'cache_exp_ibm']
get_activation = ['/usr/local/bin/ibmcloud', 'fn', 'activation', 'get']

In [4]:
pattern = re.compile('\s*[a-z0-9]+\s*blackbox')

In [5]:
def find_all_activation_id():
    p = subprocess.Popen(activation_list, stdout=subprocess.PIPE)
    ret = p.stdout.read()
    ret = ret.decode('ascii')
    ret_list = pattern.findall(ret)
    p.kill()
    return set(map(lambda i: i.split(' ')[1], ret_list))

In [6]:
# %time find_all_activation_id()

In [7]:
def invoke_new_func():
    p = subprocess.Popen(invoke_activation, stdout=subprocess.PIPE)
    ret = p.stdout.read()
    ret = ret.decode('ascii')
    p.kill()
    return ret.strip().split(' ')[-1]

In [8]:
# %time invoke_new_func()

In [9]:
def run(con=1):
    p_list = []
    for i in range(con):
        p_list.append(subprocess.Popen(invoke_activation, stdout=subprocess.PIPE))
    ret_list = []
    for p in p_list:
        ret = p.stdout.read()
        ret = ret.decode('ascii')
        ret_list.append(ret.strip().split(' ')[-1])
        p.kill()
    return set(ret_list)

In [11]:
def run_block(con=1):
    current_ids = run(con)
    sleep(215)
    while not current_ids.issubset(find_all_activation_id()):
        pass
    return current_ids

In [62]:
%time run_block(2)

fatal: bad g in signal handler


CPU times: user 469 ms, sys: 3.26 s, total: 3.73 s
Wall time: 3min 48s


{'2995e066f4cb405295e066f4cbe05278', 'db113371cd004328913371cd00a32830'}

In [12]:
def get_exp_result(activation_id, con):
    get_acti_with_id = list(get_activation)
    get_acti_with_id.append(activation_id)
    
    p = subprocess.Popen(get_acti_with_id, stdout=subprocess.PIPE)
    ret = p.stdout.read()
    ret = ret.decode('ascii')
    p.kill()
    ret = ret[ret.find('\n')+1:]
    with open('./data/activations/{}-{}.json'.format(con,activation_id), 'w') as f:
        f.write(ret)
    return ret

In [41]:
# %time ret = get_exp_result('1f006abf04124b02806abf04120b02b1', 3)

CPU times: user 2.24 ms, sys: 15.5 ms, total: 17.7 ms
Wall time: 693 ms


In [49]:
a

{1, 2, 3}

In [51]:
str(a)

'{1, 2, 3}'

In [13]:
%time find_all_activation_id()

CPU times: user 3.23 ms, sys: 18.9 ms, total: 22.1 ms
Wall time: 707 ms


{'009b5b95e56c45039b5b95e56ca50381',
 '0c6116aea3e94407a116aea3e9540739',
 '1b2f9f5d55f244a3af9f5d55f254a3a6',
 '267cdc0f42f44e35bcdc0f42f44e353f',
 '2b75c9f13059438ab5c9f13059638aa4',
 '31fa79146e6c488aba79146e6c688a88',
 '34c4dd8bc2a6430784dd8bc2a6d307fa',
 '3a210b2a35344ab1a10b2a3534fab13b',
 '3a2caab3d81b48ccacaab3d81b68cc69',
 '4259ffc3d2c24b0199ffc3d2c2fb0198',
 '4338ae7896b64f6eb8ae7896b69f6ef7',
 '49514b71b5a34b36914b71b5a39b368c',
 '5cb005b27a3d41b9b005b27a3d41b93d',
 '661cb047687346b89cb0476873c6b84e',
 '69a701c51bcb486da701c51bcb886d6d',
 '69ea93260c1d47bfaa93260c1d87bf2b',
 '6ab47c1ddd54484eb47c1ddd54284e1d',
 '79cd470effde4f158d470effde2f15f3',
 '7d0430b9c38c49d08430b9c38cc9d0a0',
 '7df3810f9cd941e2b3810f9cd951e277',
 '80a97f18f8f94c2aa97f18f8f9fc2ac2',
 '916e46b1a16d4467ae46b1a16d04678f',
 '9ec55fc0801f4da1855fc0801fcda19a',
 'b4219c4d36074501a19c4d36071501c7',
 'bba9bb21a73f4678a9bb21a73f46784b',
 'de4c25e54c6641498c25e54c66a14961',
 'e3a6aaa238384205a6aaa23838e205c8',
 

In [14]:
def write_ids(ids, con):
    output = str(ids)
    with open('./data/{}-id.txt'.format(con), 'w') as f:
        f.write(output)

# actual exp

In [15]:
def run_exp(con=30, start=1):
    old_activation_ids = find_all_activation_id()
    exp_activation_ids = []

    for i in range(start, con+1):
        print('Invoked for con={}'.format(i))
        run_block(i)
        current_ids = run_block(i)
        exp_activation_ids.append(current_ids)
#         print('Invoked for con={}'.format(1))
        print('ID:')
#         print(current_ids)
        write_ids(current_ids, i)
        print('Done\n\n')
    return exp_activation_ids

In [17]:
run_exp(30, 28)

Invoked for con=28
ID:
Done


Invoked for con=29
ID:
Done


Invoked for con=30
ID:
Done




[{'1102d4c78a5e458e82d4c78a5e058e04',
  '122436c64a284964a436c64a28496463',
  '2868b33d2e51422ba8b33d2e51422b17',
  '2cc15bf22b764e6b815bf22b76be6bc3',
  '375d7a8ab24a42109d7a8ab24a221017',
  '3ea86567a715472ba86567a715b72bdf',
  '5c4133a617dd4fe28133a617dd8fe225',
  '5f4706e7b51d42638706e7b51d62635b',
  '68a0098cb3464377a0098cb3465377d5',
  '68d631027add4b9e9631027addfb9e90',
  '756fc4ddf6e34564afc4ddf6e3a56447',
  '7d2071fb4a5145f5a071fb4a5165f55f',
  '8b972252d1a24a9d972252d1a2ea9d19',
  '8d6e19c623ff48caae19c623ff08ca1e',
  'a1db00fdfa674ecc9b00fdfa677ecc29',
  'a650453de35c451590453de35c25151e',
  'b9fe5e63cf1346e8be5e63cf1326e854',
  'bb3e5a71495d4d94be5a71495dcd941e',
  'c7cf1dcc495b40758f1dcc495b2075c5',
  'd5483a58a54345a6883a58a543a5a623',
  'da3a422472464db1ba422472468db1ba',
  'de6f61d35a2f4b8daf61d35a2f5b8d25',
  'e0eaed79ead34fc2aaed79ead36fc2a8',
  'e4766d66537d4387b66d66537d13871b',
  'f307171cf8944e3987171cf8948e3988',
  'f909388024884cd689388024883cd646',
  'fc9620e45